TODO:
- Не нужно всё переводить на регулярные выражения - для description можно использовать не внутренний текст, а исходный soup-объект, и выводить программу-ведуших и прочее с учетом тегов, а не из чистого текста
- Прочитать: https://habr.com/ru/post/349860/#Skobochnye_gruppy_gruppirovka_plyus_kvantifikatory
- extract-time можно использовать для того, чтобы указывать интервал времени (от минимального до конечного)
- Написать аналогичную функцию для поиска спикеров pop_speakers (т.е. не только находить и исключать предложение из текста)
- Сделать указание начальной и конечной дат f'На этой странице приведен список IT-мероприятий c {start} по {end}' 
- Написать тестирующую функцию для подбора регулярных выражений под определенный ключевой запрос - для поиска тем и спикеров
- Сделать более общим подход с исключением выражений - в шаблонах фраз использовать соответствующие регулярные выражения вместо словосочетаний
- Для контактов - pop_contacts

# 🐅 Парсер

Парсинг начинается со страниц, перечисленных в `pages.json`. Файл содержит необходимые сведения о структуре страниц сайта: стартовая страница, селекторы страницы, отвечающие за их описания (могут быть на стартовой `start_url` или внутренних страницах `event_urk`), количество соответствующих элементов. Если для ссылки события пока нет описания в файле `events.json`, в результате парсинга создаётся соответствующий объект. Этот файл используется для генерации карточек мероприятий в HTML, публикуемый на [matyushkin.github.io/events](https://matyushkin.github.io/events/).

In [3]:
from parser import *

def reload_modules():
    '''Инструмент тестирования и разработки:
    reload всех уже импортированных модулей'''
    modulenames = set(sys.modules) & set(globals())
    allmodules = [sys.modules[name] for name in modulenames]
    for module in allmodules:
        importlib.reload(module)

pages_checked = files.pages_checked()
print('Все необходимые поля имеются у следующих стартовых страниц: ')
for page in pages_checked:
    print(page)
                
for start_url in pages_checked:
    print(f"\nАнализируем стартовую страницу {start_url}")
    start_page = StartPage(start_url)
    actual = start_page.actual_events
    for i, event_url in enumerate(actual, 1):
        print(f'{i}. Обработка {event_url}')
        event = EventPage(event_url, start_page)
        del event.data['soup']
        files.events[event_url] = event.data

Все необходимые поля имеются у следующих стартовых страниц: 
https://events.yandex.ru/
https://it-events.com/

Анализируем стартовую страницу https://events.yandex.ru/
1. Обработка https://yandex.ru/promo/events/yace
'NoneType' object has no attribute 'group'
2. Обработка https://events.yandex.ru/events/hardware/31-july-2020
3. Обработка https://cloud.yandex.ru/events/156
4. Обработка https://cloud.yandex.ru/events/148
5. Обработка https://events.yandex.ru/events/mini-giperbaton-22-07-2020
6. Обработка https://cloud.yandex.ru/events/155
7. Обработка https://cloud.yandex.ru/events/154
8. Обработка https://yandex.ru/promo/maps/meetup/avto
9. Обработка https://events.yandex.ru/events/android-paranoid-16-07-2020
10. Обработка https://cloud.yandex.ru/events/153
11. Обработка https://cloud.yandex.ru/events/147
12. Обработка https://yandex.ru/promo/maps/onlineconf

Анализируем стартовую страницу https://it-events.com/
1. Обработка https://it-events.com/events/18810
2. Обработка https://it-eve

In [42]:
with open('files/events.json', 'w', encoding='utf-8') as events_file:
    json.dump(files.events, events_file, ensure_ascii=False)

# Анализируем и дополняем полученную информацию

Предварительно, если среди мероприятий имеются мероприятия, не обрабатываемые парсером (`events_special`) или рекламируемые (`events_promo`), обрабатываем их отдельно. Для этого помечаем их в датафрейме особым образом.

In [157]:
reload_modules()
df = pd.DataFrame.from_dict(files.events, orient='index')
df['description'] = df['description'].apply(lambda x: x, 'html.parser')

In [158]:
# Сразу выкидываем мероприятия, которые не хотим видеть
df.drop(files.bad['urls'])
for theme in files.bad['themes']:
    df = df[~df.title.str.contains(theme)]

# Соортируем по дате, оставляем актуальные
df = df.sort_values(by=['date'])
df = df[df['date'] >= handlers.current_date.isoformat()]   #! добавить проверку на время


#df['type'] = df.index.map(promo_and_special())

# Вписываем теги и типы мероприятий по картам тегов и типов
df['tags'] = df.apply(lambda x: handlers.find_spec(x, 'tags'), axis=1)
df['types'] = df.apply(lambda x: handlers.find_spec(x, 'event_types'), axis=1)


# Для фильтрации мероприятий по месяцам и добавления меток "Позже" и "∞"
df['month'] = df.date.apply(langs.date_to_month)

In [159]:
# def promo_and_special():
#     event_types = 'promo', 'special'
#     d = {}
#     for event_type in event_types:
#         event_urls = list(eval(f'files.events_{event_type}.keys()'))
#         for event_url in event_urls:
#             d[event_url] = event_type
#     return d


def fill_empty_cells():
    def emptyfill(x, t):
        try:
            if math.isnan(x):
                if t == str:
                    return ''
                elif t == list:
                    return ['']
                else:
                    return set()
            else:
                return x
        except:
            return x

    for col in df:
        try:
            t = type(df[col][df[col].notnull()][0])
            df[col] = df[col].apply(lambda x: emptyfill(x, t))
        except IndexError:
            pass
        
fill_empty_cells()

https://regex101.com/

In [160]:
_ = df.description.apply(lambda x: langs.soup_to_text(BeautifulSoup(x, 'html.parser')))

In [161]:
# Попытаемся найти дополнительно спикеров из описания докладов
df['description'] = _.apply(lambda x: x['text'])
df['themes'] += _.apply(lambda x: x['themes'])
df['speakers'] += _.apply(lambda x: x['speakers'])

In [162]:
# В строках устраним лишние пробелы
df.description = df.description.apply(lambda x: " ".join(x.split()))

# Собираем HTML-страницу и деплоим проект 💃

Для сборки страницы используем BeautifulSoup. Страницу собираем, объединяя soup-объекты в один.

In [163]:
from bs4 import BeautifulSoup, Comment
from itertools import zip_longest
import copy

with open("files/event_card_template.html") as template:
    template = BeautifulSoup(template.read(), 'html.parser')


def add_info(template, data):
    s = copy.copy(template)
    
    # Информационный блок
    data_block = s.find("div", {"class":"event"})
    companies = [*data.organizers, *data.speakers_companies]
    companies = " ".join(list(set(companies))).strip()
    data_block['data-companies'] = companies
    
    data_block['data-month'] = data.month
    data_block['data-price'] = data.price
    data_block['data-online'] = data.online_status
    if data.tags:
        data_block['data-tags'] = ", ".join(data.tags)
    else:
        data_block['data-tags'] = ""
    if data.types:
        data_block['data-types'] = ", ".join(data.types)
    else:
        data_block['data-types'] = ""
#     if data.type == 'promo':
#         data_block['class'].append('promo')
    
    # Обработка заголовка
    header = s.find("h2")
    header.string = data.title
    
    # Блок с описанием места и времени
    time_and_space = s.find("p", {"class":"time_and_space"})
    s.time.string = langs.make_datetime_string(data.date, data.time)
    s.time.wrap(s.new_tag('a', attrs={'href':data.event_url}))
    if data.reg_url:
        time_and_space.append(', ')
        reg_url = s.new_tag('a', attrs={'href':data.reg_url})
        reg_url.string = 'регистрация'
        time_and_space.append(reg_url) 
    time_and_space.append('.')
    
    # Блок с темами и докладчиками
    themes_and_speakers = s.find("p", {"class":"themes_and_speakers"})
    
    if data.themes != ['']:
        for theme in data.themes:
            if theme:
                line = s.new_tag('li')
                line.string = theme
                s.ul.append(line)
    if data.speakers != ['']:
        if len(data.speakers) == 1:
            themes_and_speakers.append(f'👤 {data.speakers[0]}.')
        else:
            speakers = [x for x in data.speakers if x]
            speakers_str = ', '.join(speakers)
            themes_and_speakers.append(f'👥 {speakers_str}.')
    if data.themes == [''] and data.speakers != ['']:
        s.ul.extract()
    if data.themes == [''] and data.speakers == ['']:
        s.find("p", {"class":"themes_and_speakers"}).extract()
    
    # Блок с описанием мероприятия
    description = s.find("p", {"class":"description"})
    description.string = data.description
    
    # Удаляем комментарии
    for element in s(text = lambda text: isinstance(text, Comment)):
        element.extract()

    return s

df['soup'] = df.apply(lambda x: add_info(template, x), axis=1)
df['html'] = df['soup'].apply(lambda x: str(x))
total = ''.join(df['html'])

total += '''<div class="filter__msg" hidden>
<p>🕵 К сожалению, для выбранных фильтров пока
нет ни одного подходящего мероприятия.</p></div>'''

path_to_html_template = "../mgio/11ty/_includes/events.njk"
with open(path_to_html_template, 'w', encoding='utf-8') as html_template:
    html_template.write(f'{total}')

Используем тот же подход для сборки меню фильтров. 

In [164]:
import langs

all_tags = set(itertools.chain.from_iterable(df.tags))
all_types = set(itertools.chain.from_iterable(df.types))
all_locs = set(itertools.chain.from_iterable(df.location))
all_months = langs.month_names_for_time_filters()[1:] + ['Позже', '∞']
#all_types = set(itertools.chain.from_iterable(df.type))

with open("files/event_filter_panel_template.html") as filter_template:
    filter_soup = BeautifulSoup(filter_template.read(), 'html.parser') 


def filter_item_append(name, value):
    s = filter_soup.find('div', {'class':f'filter__container--{name}'}).div
    t = filter_soup.new_tag('div')
    t['class'] = 'filter__item filter__button'
    t.string = value
    s.append(t)
    

for tag in all_tags:
    filter_item_append('tag', tag)

for type_ in all_types:
    filter_item_append('type', type_)

for month in all_months:
    filter_item_append('month', month)

for loc in all_locs:
    filter_item_append('location', loc)

    
with open("../mgio/11ty/_includes/events_filter.njk", 'w', encoding='utf-8') as html_template:
    html_template.write(str(filter_soup))
    
!cd ../mgio/11ty/; npx eleventy --passthroughall --output=../../matyushkin.github.io
!cd ../matyushkin.github.io/; rm -rf 404
!cd ../matyushkin.github.io/; rm -rf README


# посмотрим, что получилось в браузере
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

Writing ../../matyushkin.github.io/404/index.html from ./404.html.
Writing ../../matyushkin.github.io/README/index.html from ./README.md.
Writing ../../matyushkin.github.io/index.html from ./index.html.
Writing ../../matyushkin.github.io/cv/index.html from ./cv/index.html.
Writing ../../matyushkin.github.io/donate/index.html from ./donate/index.html.
Writing ../../matyushkin.github.io/links/index.html from ./links/index.html.
Writing ../../matyushkin.github.io/events/index.html from ./events/index.html.
Writing ../../matyushkin.github.io/posts/index.html from ./posts/index.html.
Writing ../../matyushkin.github.io/spb/index.html from ./spb/index.html.
Writing ../../matyushkin.github.io/texts/index.html from ./texts/index.html.
Copied 18 files / Wrote 10 files in 0.23 seconds (23.0ms each, v0.11.0)


True

In [165]:
!cd ../matyushkin.github.io/; git add . ; git commit -m "Better presentation of events data"; git push origin master

[master 9530964] Better presentation of events data
 1 file changed, 1 insertion(+), 2 deletions(-)
Counting objects: 4, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 481 bytes | 481.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:matyushkin/matyushkin.github.io.git
   1c75f62..9530964  master -> master
